In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Reproducibility:

In [4]:
test_split = 0.2
random = 125

In [2]:
LABELPATH = 'nih_sample_data/sample_labels.csv'

IMAGEPATH = 'nih_sample_data/images'

labels_df = pd.read_csv(LABELPATH)

labels = []

data = []

label = labels_df['Finding Labels']
paths = labels_df['Image Index']
i = 0
for p in paths:
    l = label[i]
    pfull = IMAGEPATH+"/"+p
    img = cv2.imread(pfull)
    #print(pfull)
    if(type(img) == np.ndarray):
        #print("here")
        #resize image and add to data set
        img = cv2.resize(img, (128, 128))
        data.append(img)
        #split labels and add to label set
        #TODO: one hot encoding?
        l = l.split("|")
        labels.append(l)
    else:
        print(type(img))
    i+=1

#scale pixel intensities to between 0 and 1
data = np.array(data, dtype="float64")/255.0
labels = np.array(labels)


In [3]:
print(data.shape)
print(labels.size)

(5606, 128, 128, 3)
[list(['Emphysema', 'Infiltration', 'Pleural_Thickening', 'Pneumothorax'])
 list(['Cardiomegaly', 'Emphysema']) list(['No Finding']) ...
 list(['Infiltration']) list(['No Finding']) list(['No Finding'])]


In [5]:
# convert labels from strings into binary with scikit
# two hot encoding
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)


Number of images per class:

Hernia - 13 images

Pneumonia - 62 images

Fibrosis - 84 images

Edema - 118 images

Emphysema - 127 images

Cardiomegaly - 141 images

Pleural_Thickening - 176 images

Consolidation - 226 images

Pneumothorax - 271 images


Mass - 284 images

Nodule - 313 images

Atelectasis - 508 images

Effusion - 644 images

Infiltration - 967 images

No Finding - 3044 images

We will definitely need to augment images

In [12]:
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size = test_split, random_state = random)

aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, zca_whitening=True, height_shift_range=0.1, shear_range=0.2,
                        zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/image.py:492: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


## Building the model

In [6]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [7]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes, finalAct="softmax"):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
 
        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3,3)))
        model.add(Dropout(0.25))
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3,3), padding="same", input_shape= inputShape ))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3,3), padding="same", input_shape=inputShape ))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        # use a *softmax* activation for single-label classification
        # and *sigmoid* activation for multi-label classification
        model.add(Dense(classes))
        model.add(Activation(finalAct))
        
        return model

In [9]:
IMAGE_DIM = (128, 128, 3)
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
model = SmallerVGGNet.build(
    width=128, height=128, depth=3, classes=len(mlb.classes_), finalAct="sigmoid")

opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)



In [14]:
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy", "categorical_accuracy"])

print("Beginning Training")
H = model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
    validation_data=(testX, testY),
    steps_per_epoch = len(trainX)//BS,
    epochs=EPOCHS, verbose=1)


Beginning Training
Epoch 1/75


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/image.py:571: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/preprocessing/image.py:589: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


140/140 [==============================] - 263s 2s/step - loss: 0.4335 - acc: 0.8179 - categorical_accuracy: 0.3319 - val_loss: 0.4218 - val_acc: 0.8788 - val_categorical_accuracy: 0.5160
Epoch 2/75
  5/140 [>.............................] - ETA: 4:09 - loss: 0.2496 - acc: 0.9088 - categorical_accuracy: 0.4500

KeyboardInterrupt: 